# Lab [Number]: Mastering Metrics Recreation

In this lab, we'll be recreating the findings from this [paper](http://assets.press.princeton.edu/chapters/s10363.pdf). The original analysis was done using a software called STATA. You can view the original STATA code [here](http://www.masteringmetrics.com/wp-content/uploads/2020/04/NHIS2009_hicompare_v2.do).  In this notebook, we'll being doing the same analysis using ```python``` and ```pandas``` instead. 


We'll be using data from the 2009 National Health Interview Survey (NHIS),  an annual survey of the U.S. population with detailed information on health and health insurance. Among many other things, the NHIS asks: 
*“Would you say your health in general is excellent, very good, good, fair, or poor?”* 

The NHIS uses this question data to code an index that assigns 5 to excellent health and 1 to poor health in a sample of married 2009 NHIS respondents who may or may not be insured. In this notebook, we'll be taking a look at the differences in statistics for those who have insurance, and those who do not. 

Run the next cell to import the libraries we'll be using to do our analysis

In [1]:
import pandas as pd
import numpy as np
pd.set_option('max_colwidth', None)
pd.set_option('display.max_columns', None)

## Load Data
Load the NHIS_four_drop.csv file into a ```pandas``` dataframe called ```df```.

Replace the ```...``` with the correct ```pandas``` function to read in the data.
 * If you're stuck, try reading the [documentation](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html)

In [2]:
# Run this cell to load our data
#data_string = "NHIS_four_drop.csv" 
#df = ...
#df

#####

data_string = "NHIS_four_drop.csv" 
df = pd.read_csv(data_string)
df

,year,serial,hhweight,pernum,perweight,sampweight,age,famsize,fml,nwhite,hi,yedu,empl,hlth,inc,marradult,adltempl,hi_hsb1
0,2009,3,7871,1,8938,22029.0,29,4,1,0,0,14,0,4,19282.932,1,1,NaN
1,2009,3,7871,4,8967,NaN,35,4,0,0,0,11,1,4,19282.932,1,1,0.0
2,2009,5,7871,1,8905,NaN,32,4,0,0,1,12,1,3,167844.530,1,2,1.0
3,2009,5,7871,2,8889,22190.0,34,4,1,0,1,16,1,3,167844.530,1,2,NaN
4,2009,6,7871,1,8378,19284.0,65,2,0,0,1,14,0,3,41679.344,1,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29729,2009,41173,2220,2,2496,5732.0,57,2,1,0,1,12,0,4,167844.530,1,1,NaN
29730,2009,41175,2624,1,3135,7200.0,67,2,1,0,1,14,0,3,61102.973,1,0,NaN
29731,2009,41175,2624,2,3022,NaN,68,2,0,0,1,14,0,2,61102.973,1,0,1.0
29732,2009,41176,2200,1,2532,18062.0,62,7,0,0,0,9,1,1,167844.530,1,2,0.0


Looking at the data, is there anything that looks wrong? Try running the next cell if you are unsure.

In [3]:
df[["sampweight","hi_hsb1"]].head() # this cell selects the first 5 rows of the sampweight and hi_hsb1 columns

,sampweight,hi_hsb1
0,22029.0,NaN
1,NaN,0.0
2,NaN,1.0
3,22190.0,NaN
4,19284.0,1.0


If we look at the ```hi_hsb1``` and ```sampweight``` columns, we see that there are some values of NaN. These values are called Not a Number values. This means that there are missing values that we will have to fill before we can do our analysis.

## Cleaning the Data

In this section, we will be cleaning and selecting the data in preparation for the final output table. 

In this next cell, we'll be using ```groupby```, ```sum```, and ```reset_index``` to generate some missing values. These operations are similar to the ```group``` function in the ```datascience``` package.

The cell below does the following:
* ```groupby``` and ```sum``` creates one unique row per ```serial```
* ```reset_index``` removes ```serial``` as the index of the dataframe 
* ```[["serial","hi_hsb1"]]``` selects the ```serial``` and ```hi_hsb1``` columns

In [4]:
# Run this cell to begin the cleaning process
df2=df.groupby("serial").sum().reset_index()[["serial","hi_hsb1"]]
df2.head()

,serial,hi_hsb1
0,3,0.0
1,5,1.0
2,6,1.0
3,7,1.0
4,8,1.0


In the next cell, we'll be using ```merge```, ```drop```, and ```rename```. These are similar to ```join```, ```drop```, and ```relabel``` in the ```datascience``` package.

The next cell does the following:
* ```merge``` adds on the dataframe we made in the last cell onto the main dataframe using ```serial``` as a key.
* ```drop``` removes an extra column ```hi_hsb1_x```
* ```rename``` renames the column ```hi_hsb1_y``` to ```hi_hsb``` 

In [5]:
# Run this cell to continue the data cleaning process
df3=df.merge(df2,how="left",on="serial").drop("hi_hsb1_x",axis=1).rename(columns={"hi_hsb1_y":"hi_hsb"})
df3.head()

,year,serial,hhweight,pernum,perweight,sampweight,age,famsize,fml,nwhite,hi,yedu,empl,hlth,inc,marradult,adltempl,hi_hsb
0,2009,3,7871,1,8938,22029.0,29,4,1,0,0,14,0,4,19282.932,1,1,0.0
1,2009,3,7871,4,8967,NaN,35,4,0,0,0,11,1,4,19282.932,1,1,0.0
2,2009,5,7871,1,8905,NaN,32,4,0,0,1,12,1,3,167844.530,1,2,1.0
3,2009,5,7871,2,8889,22190.0,34,4,1,0,1,16,1,3,167844.530,1,2,1.0
4,2009,6,7871,1,8378,19284.0,65,2,0,0,1,14,0,3,41679.344,1,1,1.0


## Selecting the Data

In this section, we'll be selecting the data to use in the final output.

Here's a quick refresher of selecting using ```pandas```:

### Example 1:
Selects rows with a ```serial``` of 7372

In [6]:
df[df["serial"]==7372]

,year,serial,hhweight,pernum,perweight,sampweight,age,famsize,fml,nwhite,hi,yedu,empl,hlth,inc,marradult,adltempl,hi_hsb1
5392,2009,7372,1421,1,2330,NaN,51,4,0,0,0,14,1,3,41679.344,1,2,0.0
5393,2009,7372,1421,2,1855,9373.0,47,4,1,0,0,9,1,4,41679.344,1,2,NaN


### Example 2:
Selects rows with a ```yedu``` larger than or equal to 16

In [7]:
df[df["yedu"]>=16]

,year,serial,hhweight,pernum,perweight,sampweight,age,famsize,fml,nwhite,hi,yedu,empl,hlth,inc,marradult,adltempl,hi_hsb1
3,2009,5,7871,2,8889,22190.0,34,4,1,0,1,16,1,3,167844.53,1,2,NaN
8,2009,8,8658,1,8817,NaN,60,4,0,0,1,16,0,4,167844.53,1,1,1.0
10,2009,9,12833,1,15011,NaN,61,2,1,0,1,16,0,5,167844.53,1,1,NaN
11,2009,9,12833,2,13578,37863.0,62,2,0,0,1,16,1,3,167844.53,1,1,1.0
14,2009,17,7871,1,10226,NaN,49,3,0,0,1,16,1,4,167844.53,1,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29700,2009,41115,2220,1,2582,NaN,40,4,0,0,1,16,1,4,167844.53,1,2,1.0
29701,2009,41115,2220,2,2499,NaN,36,4,1,0,1,16,1,4,167844.53,1,2,NaN
29710,2009,41140,829,1,888,2427.0,36,4,1,0,1,18,1,4,167844.53,1,2,NaN
29711,2009,41140,829,2,962,NaN,36,4,0,0,1,16,1,4,167844.53,1,2,1.0


### Example 3:
Selects rows with a ```yedu``` greater than or equal to 15 **AND** ```yedu``` less than or equal to 17

In [8]:
df[(df["yedu"]>=15) & (df["yedu"]<=17)] 

,year,serial,hhweight,pernum,perweight,sampweight,age,famsize,fml,nwhite,hi,yedu,empl,hlth,inc,marradult,adltempl,hi_hsb1
3,2009,5,7871,2,8889,22190.0,34,4,1,0,1,16,1,3,167844.530,1,2,NaN
8,2009,8,8658,1,8817,NaN,60,4,0,0,1,16,0,4,167844.530,1,1,1.0
10,2009,9,12833,1,15011,NaN,61,2,1,0,1,16,0,5,167844.530,1,1,NaN
11,2009,9,12833,2,13578,37863.0,62,2,0,0,1,16,1,3,167844.530,1,1,1.0
14,2009,17,7871,1,10226,NaN,49,3,0,0,1,16,1,4,167844.530,1,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29694,2009,41109,2775,1,4001,NaN,26,2,1,0,1,16,1,4,61102.973,1,2,NaN
29695,2009,41109,2775,2,3854,9646.0,26,2,0,0,1,16,1,4,61102.973,1,2,1.0
29700,2009,41115,2220,1,2582,NaN,40,4,0,0,1,16,1,4,167844.530,1,2,1.0
29701,2009,41115,2220,2,2499,NaN,36,4,1,0,1,16,1,4,167844.530,1,2,NaN


Select data where ```age``` is greater than or equal to 26 **AND** ```age``` is less than or equal to 59 **AND** ```marradult``` is equal to 1 **AND** ```adltempl``` is greater then or equal to 1. Assign each ```...``` to the corresponding selection.
* Hint: Take a look at the cells above for some pointers. Also remember to surround each of the filters!


Python Operators:
* Equals: ```==```
* Greater than: ```>```
* Greater than or equal to: ```>=```
* Less than: ```<```
* Less than or equal to: ```<=```
* And: ```&```

In [9]:
# Replace each ... with the corresponding selection.

#age_greater_than_or_equal_to_26 = ...
#age_less_than_or_equal_to_59 = ...
#marradult_equal_to_1 = ...
#adltempl_is_greater_than_or_equal_to_1 = ...

#data_selected = df3[age_greater_than_or_equal_to_26 & 
  #                  age_less_than_or_equal_to_59 & 
 #                   marradult_equal_to_1 & 
 #                   adltempl_is_greater_than_or_equal_to_1]
#data_selected 

##############

age_greater_than_or_equal_to_26 = df3["age"] >= 26
age_less_than_or_equal_to_59 = df3["age"]<= 59
marradult_equal_to_1 = df3["marradult"] == 1
adltempl_is_greater_than_or_equal_to_1 = df3["adltempl"] >= 1

data_selected = df3[age_greater_than_or_equal_to_26 & 
                    age_less_than_or_equal_to_59 & 
                    marradult_equal_to_1 & 
                    adltempl_is_greater_than_or_equal_to_1]
data_selected 

,year,serial,hhweight,pernum,perweight,sampweight,age,famsize,fml,nwhite,hi,yedu,empl,hlth,inc,marradult,adltempl,hi_hsb
0,2009,3,7871,1,8938,22029.0,29,4,1,0,0,14,0,4,19282.932,1,1,0.0
1,2009,3,7871,4,8967,NaN,35,4,0,0,0,11,1,4,19282.932,1,1,0.0
2,2009,5,7871,1,8905,NaN,32,4,0,0,1,12,1,3,167844.530,1,2,1.0
3,2009,5,7871,2,8889,22190.0,34,4,1,0,1,16,1,3,167844.530,1,2,1.0
7,2009,7,7871,2,8558,NaN,59,2,1,0,1,12,1,2,41679.344,1,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29722,2009,41164,928,1,1860,NaN,29,3,0,1,0,14,1,5,19282.932,1,2,0.0
29723,2009,41164,928,2,1499,NaN,28,3,1,1,0,14,1,3,19282.932,1,2,0.0
29728,2009,41173,2220,1,2654,NaN,57,2,0,0,1,12,1,4,167844.530,1,1,1.0
29729,2009,41173,2220,2,2496,5732.0,57,2,1,0,1,12,0,4,167844.530,1,1,1.0


Run the next cell to remove single person households. ```groupby``` creates a unique values for each ```serial``` value and ```filter(lambda x: len(x)>1)``` selects rows with more than one record per serial number. Don't worry if it takes a couple of seconds for the cell to run.

In [10]:
# Run this cell to remove single person households
households_only = data_selected.groupby(["serial"]).filter(lambda x: len(x)>1)
households_only

,year,serial,hhweight,pernum,perweight,sampweight,age,famsize,fml,nwhite,hi,yedu,empl,hlth,inc,marradult,adltempl,hi_hsb
0,2009,3,7871,1,8938,22029.0,29,4,1,0,0,14,0,4,19282.932,1,1,0.0
1,2009,3,7871,4,8967,NaN,35,4,0,0,0,11,1,4,19282.932,1,1,0.0
2,2009,5,7871,1,8905,NaN,32,4,0,0,1,12,1,3,167844.530,1,2,1.0
3,2009,5,7871,2,8889,22190.0,34,4,1,0,1,16,1,3,167844.530,1,2,1.0
12,2009,10,7871,1,9587,24220.0,45,2,0,0,1,12,1,4,85985.780,1,2,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29721,2009,41161,1647,2,1776,NaN,51,2,1,0,1,14,1,3,61102.973,1,2,1.0
29722,2009,41164,928,1,1860,NaN,29,3,0,1,0,14,1,5,19282.932,1,2,0.0
29723,2009,41164,928,2,1499,NaN,28,3,1,1,0,14,1,3,19282.932,1,2,0.0
29728,2009,41173,2220,1,2654,NaN,57,2,0,0,1,12,1,4,167844.530,1,1,1.0


## Processing and Formatting the Data

In this section, we'll process the data and format it like it is presented in the original paper.

In [11]:
from utils import format_data
from utils import process_data

We'll be using two functions, ```format_data``` and ```process_data```, to calculate values and display them like they are shown in Table 1.1 of the paper.

How do we use ```process_data``` and ```format_data```? Run the following cell to find out about ```process_data```. The ```?``` in ```jupyter``` shows the documentation of the function to the ```?``` follows.

In [12]:
process_data?

```process_data``` takes in a pandas dataframe (```df```) and a binary variable ```fml``` that represents if the data being processed should select males (0) or females (1). 

In [13]:
format_data?

```format_data``` takes in a python list (formatted in a way like the output of ```process_data```) and returns text and a pandas dataframe.

Let's see these functions in action!

### Example 1

Processes the data in our original dataframe (```df```), selecting rows with ```fml == 0``` (all males) 

In [14]:
processed_data_output = process_data(df, 0)
processed_data_output

[0,
 3.76,
 1.06,
 3.7,
 1.03,
 0.06,
 0.01,
 0.18,
 0.19,
 -0.01,
 0.01,
 51.66,
 41.23,
 10.43,
 0.1,
 13.81,
 11.28,
 2.53,
 0.05,
 3.15,
 3.92,
 -0.77,
 0.02,
 0.71,
 0.74,
 -0.03,
 0.01,
 90475.51,
 42208.9,
 48266.61,
 259.57,
 12680,
 2187]

### Example 2

Formats the list output by ```process_data``` in the previous cell. 

In [16]:
format_data(processed_data_output)

Husbands


,,Some HI (1),No HI (0),Difference (3)
0,Health index,3.76 [1.06],3.7 [1.03],0.06 (0.01)
1,Nonwhite,0.18,0.19,-0.01 (0.01)
2,Age,51.66,41.23,10.43 (0.1)
3,Education,13.81,11.28,2.53 (0.05)
4,Family Size,3.15,3.92,-0.77 (0.02)
5,Employed,0.71,0.74,-0.03 (0.01)
6,Family Income,90475.5,42208.9,48266.61 (259.57)
7,Sample Size,12680,2187,


Notes: This table reports average characteristics for insured and uninsured married couples in the
          2009 National Health Interview Survey (NHIS). Columns (1), (2), (4), and (5) show average characteristics
          of the group of individuals specified by the column heading. Columns (3) and (6) report the difference
          between the average characteristic for individuals with and without health insurance (HI).
          Standard deviations are in brackets; standard errorsare reported in parentheses.


Now it's your turn! Using the ```households_only``` dataframe your constructed earlier, use ```process_data``` and select all the male records.

In [19]:
# Call process_data on households_only, selecting all male records
#male_processed_data = ...
#male_processed_data

############

male_processed_data = process_data(households_only, 0)
male_processed_data

[0,
 3.98,
 0.93,
 3.7,
 1.01,
 0.28,
 0.02,
 0.2,
 0.19,
 0.01,
 0.01,
 44.16,
 41.27,
 2.89,
 0.12,
 14.13,
 11.21,
 2.92,
 0.06,
 3.55,
 4.06,
 -0.51,
 0.02,
 0.92,
 0.85,
 0.07,
 0.01,
 104002.44,
 43636.02,
 60366.42,
 294.68,
 7866,
 1529]

Now it's time to format ```male_processed_data``` using ```format_data```.

In [18]:
# Call format_data on male_processed_data
# ...
format_data(male_processed_data) # provide what this table is -- some context about column names

Husbands


,,Some HI (1),No HI (0),Difference (3)
0,Health index,3.98 [0.93],3.7 [1.01],0.28 (0.02)
1,Nonwhite,0.2,0.19,0.01 (0.01)
2,Age,44.16,41.27,2.89 (0.12)
3,Education,14.13,11.21,2.92 (0.06)
4,Family Size,3.55,4.06,-0.51 (0.02)
5,Employed,0.92,0.85,0.07 (0.01)
6,Family Income,104002,43636,60366.42 (294.68)
7,Sample Size,7866,1529,


Notes: This table reports average characteristics for insured and uninsured married couples in the
          2009 National Health Interview Survey (NHIS). Columns (1), (2), (4), and (5) show average characteristics
          of the group of individuals specified by the column heading. Columns (3) and (6) report the difference
          between the average characteristic for individuals with and without health insurance (HI).
          Standard deviations are in brackets; standard errorsare reported in parentheses.


So what did we just create? The table above has three columns: Some HI (1), No HI (0), and Difference (3). The first two columns are those husbands who have some health insurance. The second column is those husbands who have no health insurance. The third column is the difference between the two groups. 

It might be easy to use these comparisons as evidence of certain causal effects.  More often than not, however, such  comparisons are misleading. Once again the problem is other things equal, or lack thereof. Comparisons of people with and without health insurance are not apples toapples; such contrasts are apples to oranges, or worse.

Now do the same for females.

In [20]:
# Call process_data on households_only, selecting all female records
#female_processed_data = ...

# Call format_data on female_processed_data
#...

####

# Call process_data on households_only, selecting all female records
female_processed_data = process_data(households_only, 1)

# Call format_data on female_processed_data
format_data(female_processed_data)

Wives


,,Some HI (1),No HI (0),Difference (3)
0,Health index,3.99 [0.93],3.61 [1.02],0.38 (0.02)
1,Nonwhite,0.2,0.18,0.02 (0.01)
2,Age,42.15,39.52,2.63 (0.12)
3,Education,14.27,11.36,2.91 (0.06)
4,Family Size,3.55,4.07,-0.52 (0.03)
5,Employed,0.76,0.54,0.22 (0.01)
6,Family Income,103364,43641.4,59722.24 (307.41)
7,Sample Size,7950,1445,


Notes: This table reports average characteristics for insured and uninsured married couples in the
          2009 National Health Interview Survey (NHIS). Columns (1), (2), (4), and (5) show average characteristics
          of the group of individuals specified by the column heading. Columns (3) and (6) report the difference
          between the average characteristic for individuals with and without health insurance (HI).
          Standard deviations are in brackets; standard errorsare reported in parentheses.


Many of the differences in the table are large (for example, a nearly 3-year schooling gap); most are statistically precise enough to rule out the hypothesis that these discrepancies are merely chance findings. It won’t surprise you  to learn that most variables tabulated here are highly correlated with health as well as with health  insurance  status. More-educated people,for example, tend to be healthier as well asbeing overrepresented in the insured group. This may be because more-educated people exercise more, smoke less, and are more likely to wear seat belts. It stands  to reason  thatthe difference in health between insured and uninsured NHIS respondents at least partly reflects the extra schooling of the insured.

Congratulations! You have successfully recreated tables from a paper!